In [ ]:
# default_exp clone_counters

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from functools import partial
from glob import glob
from typing import Callable

import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
import xarray as xr
from skimage import measure

from py_clone_detective.utils import (
    add_scale_regionprops_table_area_measurements,
    calculate_corresponding_labels,
    calculate_overlap,
    check_channels_input_suitable_and_return_channels,
    determine_labels_across_other_images_using_centroids,
    extend_region_properties_list,
    get_all_labeled_clones_unmerged_and_merged,
    img_path_to_xarr,
    last2dims,
    lazy_props,
    reorder_df_to_put_ch_info_first,
    update_1st_coord_and_dim_of_xarr,
)

# CloneCounter Classes

## Parent Class

In [ ]:
# export
class CloneCounter:
    def __init__(
        self,
        exp_name: str,
        img_name_regex: str,
        pixel_size: float,
        tot_seg_ch: str = "C0",
    ):
        self.exp_name = exp_name
        self.img_name_regex = img_name_regex
        self.pixel_size = pixel_size
        self.tot_seg_ch = tot_seg_ch

    def add_images(self, **channel_path_globs):
        return img_path_to_xarr(
            self.img_name_regex,
            self.pixel_size,
            ch_name_for_first_dim="img_channels",
            **channel_path_globs,
        )

    def add_segmentations(
        self,
        additional_func_to_map: Callable = None,
        ad_func_kwargs: dict = None,
        **channel_path_globs,
    ):
        segmentations = img_path_to_xarr(
            self.img_name_regex,
            self.pixel_size,
            ch_name_for_first_dim="seg_channels",
            **channel_path_globs,
        )

        if additional_func_to_map is not None:
            segmentations.data = segmentations.data.map_blocks(
                additional_func_to_map, **ad_func_kwargs, dtype=np.uint16
            )

        segmentations.data = segmentations.data.map_blocks(
            last2dims(partial(measure.label)), dtype=np.uint16
        )
        return segmentations

    def combine_C0_overlaps_and_measurements(self):
        ov_df = (
            self.results_overlaps.pivot(
                index=["img_name", "C0_labels"],
                columns=["colocalisation_ch"],
                values="is_in_label",
            )
            .query("C0_labels != 0")
            .copy()
        )
        sk_df = self.results_measurements.query("seg_ch== 'C0'").set_index(
            ["seg_img", "label"]
        )
        sk_df.index.rename(["img_name", "C0_labels"], inplace=True)
        return pd.merge(ov_df, sk_df, left_index=True, right_index=True)

    def determine_seg_img_channel_pairs(
        self, seg_channels: list = None, img_channels: list = None
    ):
        seg_channels = check_channels_input_suitable_and_return_channels(
            channels=seg_channels,
            available_channels=self.image_data.seg_channels.values.tolist(),
        )

        img_channels = check_channels_input_suitable_and_return_channels(
            channels=img_channels,
            available_channels=self.image_data.img_channels.values.tolist(),
        )

        seg_img_channel_pairs = pd.DataFrame()
        seg_img_channel_pairs["image_channel"] = pd.Series(img_channels)
        seg_img_channel_pairs["segmentation_channel"] = pd.Series(seg_channels)
        self.seg_img_channel_pairs = seg_img_channel_pairs.fillna(method="ffill")[
            ["segmentation_channel", "image_channel"]
        ]

    def make_measurements(
        self,
        seg_channels: list = None,
        img_channels: list = None,
        extra_properties: list = None,
        **kwargs,
    ):

        self.determine_seg_img_channel_pairs(seg_channels, img_channels)

        properties = extend_region_properties_list(extra_properties)

        results = list()
        for _, seg_ch, img_ch in self.seg_img_channel_pairs.itertuples():
            for seg, img in zip(
                self.image_data["segmentations"].loc[seg_ch],
                self.image_data["images"].loc[img_ch],
            ):
                results.append(
                    lazy_props(
                        seg.data,
                        img.data,
                        seg.seg_channels.item(),
                        img.img_channels.item(),
                        seg.img_name.item(),
                        img.img_name.item(),
                        properties,
                        **kwargs,
                    )
                )

        df = dd.from_delayed(results).compute()
        df = add_scale_regionprops_table_area_measurements(df, self.pixel_size)
        self.results_measurements = reorder_df_to_put_ch_info_first(df)
        self._determine_max_seg_label_levels()

    def _determine_max_seg_label_levels(self):
        self.tot_seg_ch_max_labels = (
            self.image_data["segmentations"]
            .loc[self.tot_seg_ch]
            .data.map_blocks(
                lambda x: np.unique(x).shape[0], drop_axis=(1, 2), dtype=np.uint16,
            )
            .compute()
            .max()
        )

    def _create_df_from_arr(self, arr):
        return (
            xr.DataArray(
                np.moveaxis(arr, 1, 0),
                coords=(
                    self.image_data["segmentations"].coords["seg_channels"][1:],
                    self.image_data["segmentations"].coords["img_name"],
                    np.arange(self.tot_seg_ch_max_labels),
                ),
                dims=("colocalisation_ch", "img_name", "C0_labels",),
            )
            .to_dataframe("is_in_label")
            .reset_index()
            .dropna()
        )

    def measure_overlap(self):
        self._determine_max_seg_label_levels()
        arr = (
            self.image_data["segmentations"]
            .data.map_blocks(
                calculate_overlap,
                drop_axis=[0],
                dtype=np.float64,
                num_of_segs=self.image_data["segmentations"].shape[0],
                preallocate_value=self.tot_seg_ch_max_labels,
            )
            .compute()
        )

        df = self._create_df_from_arr(arr)
        df["is_in_label"] = df["is_in_label"].astype(np.uint16)
        self.results_overlaps = df[
            ["img_name", "C0_labels", "colocalisation_ch", "is_in_label"]
        ]

    def clones_to_keep_as_dict(self, query_for_pd: str):
        return (
            self.results_measurements.query(query_for_pd)
            .groupby("int_img")
            .agg({"label": lambda x: list(x)})["label"]
            .to_dict()
        )

    def get_centroids_list(self):
        df = self.results_measurements.query("int_img_ch == @self.tot_seg_ch")
        centroids_list = list()
        for img_name in df["int_img"].unique():
            centroids_list.append(
                (
                    df.query("int_img == @img_name")
                    .loc[:, ["centroid-0", "centroid-1"]]
                    .values.astype(int)
                )
            )
        return centroids_list

    def add_clones_and_neighbouring_labels(
        self,
        query_for_pd: str = 'int_img_ch == "C1" & mean_intensity > 1000',
        name_for_query: str = "C1",
        calc_clones: bool = True,
    ):
        new_coord = [
            "extended_tot_seg_labels",
            "total_neighbour_counts",
            f"{name_for_query}pos_neigh_counts",
            f"{name_for_query}neg_neigh_counts",
        ]

        if calc_clones:
            new_coord.append("clone")

        clone_coords, clone_dims = update_1st_coord_and_dim_of_xarr(
            self.image_data["images"],
            new_coord=new_coord,
            new_dim=f"{name_for_query}_neighbours",
        )

        clones_to_keep = self.clones_to_keep_as_dict(query_for_pd)

        new_label_imgs = get_all_labeled_clones_unmerged_and_merged(
            self.image_data["segmentations"].loc[self.tot_seg_ch],
            clones_to_keep,
            calc_clones,
        )

        return xr.DataArray(
            data=new_label_imgs,
            coords=clone_coords,
            dims=clone_dims,
            attrs={f"{self.tot_seg_ch}_labels_kept_query": query_for_pd},
        )

    def colabels_to_df(self, colabels, name_for_query):
        return (
            xr.DataArray(
                colabels,
                coords=(
                    self.image_data[name_for_query].coords[
                        f"{name_for_query}_neighbours"
                    ],
                    foo.image_data[name_for_query].coords["img_name"],
                    range(1, colabels.shape[2] + 1),
                ),
                dims=(f"{name_for_query}_neighbours", "img_name", "labels"),
            )
            .to_dataframe("colabel")
            .reset_index()
            .dropna()
            .pivot(
                index=["img_name", "labels"],
                columns=[f"{name_for_query}_neighbours"],
                values="colabel",
            )
            .astype(np.uint16)
        )

    def measure_clones_and_neighbouring_labels(self, name_for_query):
        self.get_centroids_list()
        colabels = calculate_corresponding_labels(
            self.image_data[name_for_query].data,
            self.get_centroids_list(),
            self.image_data[name_for_query].shape[0],
            foo.tot_seg_ch_max_labels,
        )

        if not hasattr(self, "results_clones_and_neighbour_counts"):
            self.results_clones_and_neighbour_counts = dict()

        self.results_clones_and_neighbour_counts[name_for_query] = self.colabels_to_df(
            colabels, name_for_query
        )

## CloneCounter subclasses

In [ ]:
# export
class LazyCloneCounter(CloneCounter):
    def __init__(self, exp_name: str, img_name_regex: str, pixel_size: float):
        super().__init__(exp_name, img_name_regex, pixel_size)

    def add_images(self, **channel_path_globs):
        self.image_data = xr.Dataset(
            {"images": super().add_images(**channel_path_globs)}
        )

    def add_segmentations(
        self,
        additional_func_to_map: Callable = None,
        ad_func_kwargs: dict = None,
        **channel_path_globs
    ):
        self.image_data["segmentations"] = super().add_segmentations(
            additional_func_to_map, ad_func_kwargs, **channel_path_globs
        )

    def add_clones_and_neighbouring_labels(
        self,
        query_for_pd: str = 'int_img_ch == "C1" & mean_intensity > 1000',
        name_for_query: str = "filt_C1_intensity",
        calc_clones: bool = True,
    ):
        self.image_data[name_for_query] = super().add_clones_and_neighbouring_labels(
            query_for_pd, name_for_query, calc_clones
        )

In [ ]:
# export
class PersistentCloneCounter(CloneCounter):
    def __init__(self, exp_name: str, img_name_regex: str, pixel_size: float):
        super().__init__(exp_name, img_name_regex, pixel_size)

    def add_images(self, **channel_path_globs):
        self.image_data = xr.Dataset(
            {"images": super().add_images(**channel_path_globs)}
        ).persist()

    def add_segmentations(
        self,
        additional_func_to_map: Callable = None,
        ad_func_kwargs: dict = None,
        **channel_path_globs,
    ):
        self.image_data["segmentations"] = (
            super()
            .add_segmentations(
                additional_func_to_map, ad_func_kwargs, **channel_path_globs
            )
            .persist()
        )

    def add_clones_and_neighbouring_labels(
        self,
        query_for_pd: str = 'int_img_ch == "C1" & mean_intensity > 1000',
        name_for_query: str = "filt_C1_intensity",
        calc_clones: bool = True,
    ):
        self.image_data[name_for_query] = (
            super()
            .add_clones_and_neighbouring_labels(
                query_for_pd, name_for_query, calc_clones
            )
            .persist()
        )

In [ ]:
# hide
from dask.distributed import Client

c = Client()
c

/Users/ottomorris/miniconda3/envs/py_clone_det/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52328 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:52328/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:52328/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:52329,Workers: 4
Dashboard: http://127.0.0.1:52328/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:52341,Total threads: 2
Dashboard: http://127.0.0.1:52345/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:52333,


## Example using LazyCloneCounter with measure_overlap

In [ ]:
foo = LazyCloneCounter("Marcm2a_E7F1", r"a\dg\d\dp\d", 0.275)

foo.add_images(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_imgs/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_imgs/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_imgs/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_imgs/*.tif*",
)

foo.add_segmentations(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_label_imgs_combined_C3/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_binaries/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_label_imgs_v2/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_label_imgs/*.tif*",
)
foo.make_measurements(extra_properties=["convex_area"],)
foo.measure_overlap()
foo.combine_C0_overlaps_and_measurements()

## Example using LazyCloneCounter with add_clones_and_neighbouring_labels

In [ ]:
from skimage import morphology

In [ ]:
foo = PersistentCloneCounter("Marcm2a_E7F1", r"a\dg\d\dp\d", 0.275)

foo.add_images(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_imgs/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_imgs/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_imgs/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_imgs/*.tif*",
)

In [ ]:
foo.add_segmentations(
    morphology.remove_small_objects,
    ad_func_kwargs={"min_size": 49},
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_label_imgs_combined_C3/*.tif*",
)

In [ ]:
foo.make_measurements()

In [ ]:
foo.results_measurements = foo.results_measurements.eval(
    "total_intensity = mean_intensity * area"
)

In [ ]:
foo.add_clones_and_neighbouring_labels(
    query_for_pd='int_img_ch == "C1" & mean_intensity > 1000',
    name_for_query="C1",
    calc_clones=True,
)

In [ ]:
foo.add_clones_and_neighbouring_labels(
    query_for_pd='int_img_ch == "C2" & total_intensity > 2e5',
    name_for_query="C2",
    calc_clones=False,
)

In [ ]:
foo.measure_clones_and_neighbouring_labels(name_for_query="C1")

In [ ]:
foo.measure_clones_and_neighbouring_labels(name_for_query="C2")

In [ ]:
foo.results_clones_and_neighbour_counts["C1"]

C1_neighbours    C1neg_neigh_counts  C1pos_neigh_counts  clone  \
img_name labels                                                  
a1g01p1  1                        0                   0      0   
         2                        0                   0      0   
         3                        0                   0      0   
         4                        0                   0      0   
         5                        0                   0      0   
...                             ...                 ...    ...   
a2g13p3  237                      0                   0      0   
         238                      0                   0      0   
         239                      0                   0      0   
         240                      0                   0      0   
         241                      0                   0      0   

C1_neighbours    extended_tot_seg_labels  total_neighbour_counts  
img_name labels                                                   
a1g01p1  1                             0                       0  
         2                             0                       0  
         3                             0                       0  
         4                             0                       0  
         5                             0                       0  
...                                  ...                     ...  
a2g13p3  237                           0                       0  
         238                           0                       0  
         239                           0                       0  
         240                           0                       0  
         241                           0                       0  

[20202 rows x 5 columns]

In [ ]:
foo.results_measurements.set_index(['int_img', 'label'])

seg_ch int_img_ch  seg_img  area  mean_intensity  centroid-0  \
int_img label                                                                
a1g01p1 1         C0         C0  a1g01p1   353     1143.484419    5.025496   
        2         C0         C0  a1g01p1   613     2521.355628    8.014682   
        3         C0         C0  a1g01p1   275     2570.625455    5.941818   
        4         C0         C0  a1g01p1   700     2873.265714   14.550000   
        5         C0         C0  a1g01p1   209     2537.277512    7.693780   
...              ...        ...      ...   ...             ...         ...   
a2g13p3 237       C0         C3  a2g13p3   712       96.773876  792.443820   
        238       C0         C3  a2g13p3   308       73.847403  793.714286   
        239       C0         C3  a2g13p3   678       95.244838  799.069322   
        240       C0         C3  a2g13p3   672       31.366071  798.055060   
        241       C0         C3  a2g13p3   212       50.495283  805.608491   

               centroid-1   area_um2  total_intensity  
int_img label                                          
a1g01p1 1       97.269122  26.695625         403650.0  
        2      222.654160  46.358125        1545591.0  
        3      383.338182  20.796875         706922.0  
        4      519.455714  52.937500        2011286.0  
        5      644.760766  15.805625         530291.0  
...                   ...        ...              ...  
a2g13p3 237    551.751404  53.845000          68903.0  
        238    459.652597  23.292500          22745.0  
        239    500.057522  51.273750          64576.0  
        240    786.084821  50.820000          21078.0  
        241    636.231132  16.032500          10705.0  

[80808 rows x 9 columns]

In [ ]:
foo.results_clones_and_neighbour_counts["C1"].query(
    "labels == extended_tot_seg_labels"
)

C1_neighbours    C1neg_neigh_counts  C1pos_neigh_counts  clone  \
img_name labels                                                  
a1g01p1  12                       2                   0      0   
         13                       3                   0      0   
         14                       5                   0      0   
         15                       2                   0      0   
         20                       3                   0      0   
...                             ...                 ...    ...   
a2g13p3  227                      3                   0      0   
         228                      3                   0      0   
         230                      5                   0      0   
         232                      3                   0      0   
         233                      2                   0      0   

C1_neighbours    extended_tot_seg_labels  total_neighbour_counts  
img_name labels                                                   
a1g01p1  12                           12                       3  
         13                           13                       3  
         14                           14                       5  
         15                           15                       2  
         20                           20                       3  
...                                  ...                     ...  
a2g13p3  227                         227                       3  
         228                         228                       3  
         230                         230                       5  
         232                         232                       3  
         233                         233                       2  

[16781 rows x 5 columns]

In [ ]:
foo.results_clones_and_neighbour_counts["C1"].query(
    "extended_tot_seg_labels != 0"
)

C1_neighbours    C1neg_neigh_counts  C1pos_neigh_counts  clone  \
img_name labels                                                  
a1g01p1  12                       2                   0      0   
         13                       3                   0      0   
         14                       5                   0      0   
         15                       2                   0      0   
         20                       3                   0      0   
...                             ...                 ...    ...   
a2g13p3  227                      3                   0      0   
         228                      3                   0      0   
         230                      5                   0      0   
         232                      3                   0      0   
         233                      2                   0      0   

C1_neighbours    extended_tot_seg_labels  total_neighbour_counts  
img_name labels                                                   
a1g01p1  12                           12                       3  
         13                           13                       3  
         14                           14                       5  
         15                           15                       2  
         20                           20                       3  
...                                  ...                     ...  
a2g13p3  227                         227                       3  
         228                         228                       3  
         230                         230                       5  
         232                         232                       3  
         233                         233                       2  

[16789 rows x 5 columns]

In [ ]:
%gui qt5

In [ ]:
import napari

view = napari.Viewer()

In [ ]:
lab_C1 = foo.image_data["C1"].sel(img_name="a1g03p2").compute()

In [ ]:
seg_C1 = foo.image_data["segmentations"].sel(img_name="a1g03p2").compute()

In [ ]:
view.add_labels(seg_C1)

/Users/ottomorris/miniconda3/envs/py_clone_det/lib/python3.8/site-packages/napari/_vispy/vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


<Labels layer 'seg_C1' at 0x7fccdda6ea90>

In [ ]:
view.add_labels(lab_C1)

<Labels layer 'lab_C1' at 0x7fccdeded7f0>

In [ ]:
view.add_labels(seg_C1 == 251)

<Labels layer 'Labels' at 0x7fccdeded310>

In [ ]:
df = foo.results_clones_and_neighbour_counts.reset_index().copy()

In [ ]:
df = df.query("extended_tot_seg_labels != 0")[
    [
        "extended_tot_seg_labels",
        "img_name",
        "total_neighbour_counts",
        "inside_clone_neighbour_counts",
        "outside_clone_neighbour_counts",
    ]
]

In [ ]:
df

In [ ]:
bar = foo.results_measurements.copy()

In [ ]:
pd.merge(
    bar,
    df,
    how="inner",
    left_on=["int_img", "label"],
    right_on=["img_name", "extended_tot_seg_labels"],
).drop(columns=["img_name", "extended_tot_seg_labels"])